In [1]:
#import funconnect
import funconnect

Connecting celiib@10.28.0.34:3306


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import datajoint as dj
from funconnect import connectomics as cncs, ta3, nda
from funconnect.connectomics import *
from funconnect import nda
import numpy as np
import pandas as pd
import pycircstat as circ
cncs.schema.spawn_missing_classes()

In [3]:
#1) Found all of the axon segments

#gets the segments id's of all of the segments that are axons
#ta3.Neurite() just has the segment_id and whether it is a dendrite or axon
axon_segments = (ta3.Neurite() & "neurite_type='axon'").proj(presyn="segment_id")
axon_segments

segmentation segmentation id,presyn segment id unique within each Segmentation
1,31
1,71
1,108
1,184
1,269
1,319
1,367


In [4]:
#ta3.Segment : has segment id and previous segment id
#ta3.Soma : has the semgnet id of the soma and the EM id
#nda.Trace : EM id and the calcium trace
post_trace_segments = (ta3.Segment & nda.Trace & ta3.Soma).proj(postsyn = 'segment_id')

In [5]:
#Result: has the postsyn targets that have both a soma and have a trace associated with them
post_trace_segments

segmentation segmentation id,postsyn segment id unique within each Segmentation
1,36456160
1,76859794
1,83286327
1,36706215
1,24927722
1,31978704
1,34028137


In [6]:
#keeps all the traces
(ta3.Segment & nda.Trace) & ta3.Soma

segmentation segmentation id,segment_id segment id unique within each Segmentation,prev_segment_id segment id in previous Segmentation version
1,36456160,27508708
1,76859794,22270104
1,83286327,27668619
1,36706215,27268097
1,24927722,28262804
1,31978704,31978704
1,34028137,25633178


In [7]:
ta3.Synapse()

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),syn_bbox_x1 (EM voxels) - bounding box,syn_bbox_y1 (EM voxels) - bounding box,syn_bbox_z1 (EM voxels) - bounding box,syn_bbox_x2 (EM voxels) - bounding box,syn_bbox_y2 (EM voxels) - bounding box,syn_bbox_z2 (EM voxels) - bounding box
1,1,42150,37421,10241,8252,461,10240,8219,458,10246,8288,465
1,4,77737,74402,10311,8449,859,10246,8375,851,10364,8529,868
1,5,70880,75780,10294,8338,794,10247,8293,791,10354,8385,798
1,6,4366217,74402,10309,8317,852,10247,8248,850,10357,8385,855
1,7,16149136,13323,10317,8594,166,10247,8542,161,10402,8653,173
1,8,58123,60926,10279,8109,694,10248,8101,689,10312,8119,700
1,9,56707,143982,10302,8374,642,10248,8345,635,10366,8434,652


In [21]:
#ta3.Synapse() has all of the synapse data: id, presynaptic, post-synaptic, coordinates and bounding box coordinates
#axon_segments --> restricts so only valid axons can be the presynaptic targets
#post_trace_segments --> restricts to only post-synaptic targets that have a valid soma and calcium trace
axon_trace_synapses = ta3.Synapse() & axon_segments & post_trace_segments
axon_trace_synapses

#what it outputs:
"""
Less synapses than the total synapse tables be restricted by only axons as presynaptic sources and only valid traces as post synaptic targets

"""

#Example: 54101819 has 5 different synapses, 2 on 36456160 and 3 on 23683777

(axon_trace_synapses & 'presyn=54101819')

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),syn_bbox_x1 (EM voxels) - bounding box,syn_bbox_y1 (EM voxels) - bounding box,syn_bbox_z1 (EM voxels) - bounding box,syn_bbox_x2 (EM voxels) - bounding box,syn_bbox_y2 (EM voxels) - bounding box,syn_bbox_z2 (EM voxels) - bounding box
1,21460,54101819,36456160,10341,21973,833,10324,21946,832,10359,22007,835
1,23689,54101819,36456160,10902,22185,828,10860,22135,826,10940,22225,832
1,73847,54101819,36456160,11638,22620,750,11585,22573,743,11665,22664,754
1,73682,54101819,23683777,11530,22770,796,11468,22724,790,11586,22828,803
1,74337,54101819,23683777,11994,22682,742,11907,22622,737,12070,22746,748


In [22]:
"""  Table explaination
dj.U part goes and gets all of the entities that have a value called presyn and postsyn but it can't be used alone and must come with other table or aggr
the .aggr part takes all presyn and postsyn pairs and counts how many total synapses there are with those pairing of presyn and postsyn and 
adds up the number of a certain axon synapses a certain post-synaptic target

"""

axons_from_unique_axon_trace_pairs_table = dj.U('presyn','postsyn').aggr(axon_trace_synapses, n_syn='count(*)')
#(axons_from_unique_axon_trace_pairs_table & "n_syn=0")
axons_from_unique_axon_trace_pairs_table & 'presyn=54101819'

"""
Conclusion: 
Yields only the axons with a unique trace and their synaptic targets with the number of targets for each segment pairing

"""


#axons_from_unique_axon_trace_pairs_table & 'n_syn=2'
axons_from_unique_axon_trace_pairs_table & 'presyn=54101819'


presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,n_syn calculated attribute
54101819,23683777,2
54101819,36456160,3


In [24]:
#should get all the presynaptic segment ids that are axons (no repeats of axons)

unique_axons = dj.U('presyn') & axons_from_unique_axon_trace_pairs_table
#unique_axons & 'presyn=54101819'

In [11]:
scan_idx = 3
num_postsyns = 4
von_r2_min = 0.05
von_pvalue_max = 0.05

In [12]:
is_tuned = 'von_r2>{} and von_pvalue<{}'.format(str(von_r2_min),str(von_pvalue_max))

In [13]:
#gets the directional preferences of only significanly tuned neurons and gets their postsyn id
functional_data = (nda.VonMises & is_tuned).proj('von_pref',postsyn='segment_id') & 'scan_idx={}'.format(str(scan_idx))
functional_data

scan_idx scan ID,segmentation segmentation id,postsyn segment id unique within each Segmentation,von_pref preferred directions
3,1,58281524,4.01851
3,1,77127834,4.7786
3,1,29789967,2.29286
3,1,59033263,0.15092
3,1,78665518,2.90859
3,1,37086820,2.477
3,1,61375585,2.36653


In [14]:
#gets just the names of the unique significantly tuned post-synaptic segments
postsyns = (dj.U('postsyn') & functional_data)
postsyns

postsyn segment id unique within each Segmentation
58281524
77127834
29789967
59033263
78665518
37086820
61375585


In [15]:
#only get synpases that correspond to the unique axons (as the presynaptic) and unique post_synaptic targets that have functional data
Synapses = ta3.Synapse() & unique_axons & postsyns
Synapses

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),syn_bbox_x1 (EM voxels) - bounding box,syn_bbox_y1 (EM voxels) - bounding box,syn_bbox_z1 (EM voxels) - bounding box,syn_bbox_x2 (EM voxels) - bounding box,syn_bbox_y2 (EM voxels) - bounding box,syn_bbox_z2 (EM voxels) - bounding box
1,412,4151099,82251040,10406,8679,298,10377,8655,295,10437,8710,302
1,629,21736,82251040,10548,8323,325,10494,8280,318,10599,8380,332
1,725,4172843,82251040,10575,8717,280,10526,8697,275,10635,8736,288
1,759,9117,82251040,10612,8407,252,10577,8381,250,10649,8436,254
1,859,18695,82251040,10705,8290,232,10656,8247,228,10753,8325,235
1,1319,144401,82251040,10951,7972,281,10910,7932,279,10991,8022,283
1,2342,31276,82251040,10441,9129,349,10416,9091,345,10474,9168,354


In [16]:
ta3.SynapseCompartmentLabel()

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment
1,21167,2
1,21174,2
1,21911,2
1,22071,2
1,22077,2
1,22089,2
1,22158,2


In [17]:
#ONLY KEEPS THE SYNAPSES THAT HAVE A LABEL AS WELL
rel_labeled_synapses = Synapses & ta3.SynapseCompartmentLabel
rel_labeled_synapses

segmentation segmentation id,synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),syn_bbox_x1 (EM voxels) - bounding box,syn_bbox_y1 (EM voxels) - bounding box,syn_bbox_z1 (EM voxels) - bounding box,syn_bbox_x2 (EM voxels) - bounding box,syn_bbox_y2 (EM voxels) - bounding box,syn_bbox_z2 (EM voxels) - bounding box
1,46320,118499244,81578447,10286,39984,176,10242,39903,172,10349,40084,182
1,47175,111200117,81578447,10778,40214,164,10746,40183,162,10808,40249,167
1,47801,122212428,81578447,10306,41287,197,10253,41176,183,10345,41379,214
1,47806,129873028,81578447,10285,41470,267,10255,41457,263,10317,41481,273
1,48047,118795109,81578447,10340,40975,216,10322,40951,213,10356,40998,218
1,48352,122217757,81578447,10564,40767,246,10526,40743,243,10598,40792,251
1,48421,119180012,81578447,10628,41368,303,10580,41325,299,10674,41411,308


In [18]:
#what happens when aggr command but don't have  a command following it
"""
1) First part makes the columns as primary keys form the rel_labeled_synapses (essnetially turning their non-primary to primary keys)
2) then combines the postsynaptic labels that match the first part as the table

"""
synaptic_label_relation = (dj.U('presyn','postsyn','synapse_id','synapse_x','synapse_y','synapse_z').aggr(rel_labeled_synapses)
                           * dj.U('synapse_id','postsynaptic_label').aggr(ta3.SynapseCompartmentLabel))
synaptic_label_relation

synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),postsynaptic_label numeric label of the compartment
412,4151099,82251040,10406,8679,298,3
629,21736,82251040,10548,8323,325,3
725,4172843,82251040,10575,8717,280,3
759,9117,82251040,10612,8407,252,3
859,18695,82251040,10705,8290,232,3
1319,144401,82251040,10951,7972,281,3
2342,31276,82251040,10441,9129,349,3


In [25]:
ta3.Soma()

OperationalError: (2006, "MySQL server has gone away (BrokenPipeError(32, 'Broken pipe'))")

InterfaceError: (0, '')

In [19]:
soma_info = (ta3.Soma() & (nda.Trace & 'scan_idx={}'.format(str(scan_idx)) )).proj('soma_x','soma_y','soma_z',postsyn='segment_id')
soma_info

segmentation segmentation id,postsyn segment id unique within each Segmentation,soma_x x location of soma in 3.58nm grid coordinates,soma_y y location of soma in 3.58nm grid coordinates,soma_z z location of soma in 3.58nm grid coordinates
1,694582,19368,8433,970
1,942272,10000,27000,0
1,10914339,53634,11008,482
1,17660351,43441,9613,895
1,18568753,44342,19691,755
1,18572815,41453,23667,481
1,20410486,19128,14358,509


In [20]:
source_table = ((synaptic_label_relation * functional_data * soma_info).proj('von_pref','soma_x','soma_y','soma_z', 
                            x_dist='synapse_x-soma_x',y_dist='synapse_y-soma_y',
            norm='sqrt(POWER((synapse_x-soma_x),2) + POWER((synapse_y-soma_y),2) + POWER((synapse_z-soma_z),2)  )'))
source_table

synapse_id synapse index within the segmentation,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,synapse_x (EM voxels),synapse_y (EM voxels),synapse_z (EM voxels),postsynaptic_label numeric label of the compartment,scan_idx scan ID,segmentation segmentation id,von_pref preferred directions,soma_x x location of soma in 3.58nm grid coordinates,soma_y y location of soma in 3.58nm grid coordinates,soma_z z location of soma in 3.58nm grid coordinates,x_dist calculated attribute,y_dist calculated attribute,norm calculated attribute
460293,35776627,78665518,20233,20156,275,3,3,1,2.90859,33076,30442,183,-12843,-10286,16454.5710670318
1384657,71999751,78665518,39775,22424,720,3,3,1,2.90859,33076,30442,183,6699,-8018,10461.992831196167
1400405,101966310,78665518,39458,34632,903,3,3,1,2.90859,33076,30442,183,6382,4190,7668.404266860218
1061050,78747675,78665518,31896,31802,217,5,3,1,2.90859,33076,30442,183,-1180,1360,1800.8764532860105
471141,74484335,78665518,20185,28775,225,3,3,1,2.90859,33076,30442,183,-12891,-1667,12998.40505600591
1423303,33344630,78665518,40370,16937,988,3,3,1,2.90859,33076,30442,183,7294,-13505,15369.954001232405
820867,108939139,78665518,27209,35686,630,3,3,1,2.90859,33076,30442,183,-5867,5244,7881.689793438968
